In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score as model_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
animals = pd.read_csv("../input/train/train.csv")
test = pd.read_csv("../input/test/test.csv")
# Any results you write to the current directory are saved as output.
animals.head(3)



    Type - Type of animal (1 = Dog, 2 = Cat)
    Name - Name of pet (Empty if not named)
    Age - Age of pet when listed, in months
    Breed1 - Primary breed of pet (Refer to BreedLabels dictionary)
    Breed2 - Secondary breed of pet, if pet is of mixed breed (Refer to BreedLabels dictionary)
    Gender - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents group of pets)
    Color1 - Color 1 of pet (Refer to ColorLabels dictionary)
    Color2 - Color 2 of pet (Refer to ColorLabels dictionary)
    Color3 - Color 3 of pet (Refer to ColorLabels dictionary)
    MaturitySize - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)
    FurLength - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)
    Vaccinated - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)
    Dewormed - Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)
    Sterilized - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)
    Health - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)
    Quantity - Number of pets represented in profile
    Fee - Adoption fee (0 = Free)
    State - State location in Malaysia (Refer to StateLabels dictionary)
    RescuerID - Unique hash ID of rescuer
    VideoAmt - Total uploaded videos for this pet
    Description - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.
    PetID - Unique hash ID of pet profile
    PhotoAmt - Total uploaded photos for this pet
    AdoptionSpeed - Categorical speed of adoption. Lower is faster. This is the value to predict. See below section for more info.



# Selección de Datos

In [ ]:
model_data = animals.drop(['AdoptionSpeed', 'Name', 'State', 'RescuerID', 'Description', 'PetID'], axis=1)
test_data = animals['AdoptionSpeed']
x_train, x_test, y_train, y_test = train_test_split(model_data, test_data, test_size=0.20, random_state=100)

En base a las investigaciones anteriores se utilizarán todas las variables con alguna representación numérica para entrenar los distintos modelos. Esto excluye el nombre, el estado, el id del rescatador, la descripción y el id de la mascota. Tambíen se exluirá la velocidad de adopción, pues esta será la variable de respuesta.

# Redes Neuronales

A continuación se modificará la variable de respuesta, utilizando la función *to_categorical* de keras, la cual crea 5 variables dummy para representar con números binarios a qué categorías de la velocidad de adopción pertenece cada fila del set de datos. Con esa nueva representación de la variable de respuesta se utilizarán dos modelos de redes neuronales distintos para predecirla.

### Red de una capa oculta utilizando la función softmax como función de activación

In [ ]:
model = Sequential()
model.add(Dense(5, input_dim=18))
model.add(Activation('softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

dummies = to_categorical(y_train, num_classes=5)
model.fit(x_train, dummies, epochs=10, batch_size=5)

score = model.evaluate(x_test, to_categorical(y_test, num_classes=5), batch_size=5)
score

#### Matriz de confusión

In [ ]:
pred = model.predict_classes(x_test)
confusion_matrix(pred, y_test)

### Red de 2 capas ocultas utilizando las funciones de activación de unidad lineal rectificada (ReLU) y sigmoide

In [ ]:
model = Sequential()
model.add(Dense(5, input_dim=18))
model.add(Activation('relu'))
model.add(Dense(5, input_dim=18))
model.add(Activation('sigmoid'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

dummies = to_categorical(y_train, num_classes=5)
model.fit(x_train, dummies, epochs=10, batch_size=5)

score = model.evaluate(x_test, to_categorical(y_test, num_classes=5), batch_size=5)
score

#### Matriz de confusión

In [ ]:
pred = model.predict_classes(x_test)
confusion_matrix(pred, y_test)

Comparando ambos modelos de redes neuronales se puede observar que el modelo de una capa con la función de activación softmax fue más efectivo que el modelo de dos capas con las funciones de activación relu y sigmoide. En las matrices de confución se observa que el modelo de una capa logró predecir valores de varias categorías, mientras que el modelo de dos capas se especializó en predecir únicamente una, pues este identificó a casi todos los datos como parte de dicha categoría. Adicionalmente, el modelo de una capa fue mucho más consistente que el otro modelo, pues ambos se corrieron varias veces con una cantidad de épocas diferente y el modelo de dos capas llego a diverger en varios casos, en lugar de converger a un único puntaje de precisión; en contraste el modelo de una capa se mantuvo en un rango de 0.25 y 0.30. Es posible a que esto se deba por la complejidad que se crea al mezclar dos funciones de activación distintas en dos capas distintas.

Comparando los modelos de redes neuronales con los modelos de investigaciones anteriores se puede concluir que estos fueron mejores que casi todos los anteriores, excepto por el modelo de regresión logística, el cual resultó tener un puntaje de precisión muy similar que el de las redes neuronales. En cuanto a su tiempo de ejecución, los modelos de redes neuronales se tardaron mucho más en llegar a una predicción que cualquiera de los modelos anteriores.

# SVM para la predicción de tipo de animal(azul= gatos, verde=perros) según género y cantidad

In [ ]:
data_x = animals[['Gender','Quantity']]
data_y = animals[['Type']]

#Original
fig, axs = plt.subplots(2, 2,figsize=(50, 50))
axs[0,0].scatter(data_x.iloc[:,0], data_x.iloc[:,1],c=data_y.values[:, 0], s=1000, cmap = ListedColormap(['blue', 'green']))
axs[0,0].set(xlabel='Gender',ylabel='Quantity', title="Original")

#Lineal
model = SVC(kernel='linear').fit(data_x,data_y)
pred = model.predict(data_x)
X1, X2 = np.meshgrid(np.arange(data_x.values[:, 0].min() - 1,data_x.values[:, 0].max() + 1),np.arange(data_x.values[:, 1].min() - 1,data_x.values[:, 1].max() + 1))
axs[0,1].contourf(X1, X2, model.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),alpha = 0.75, cmap = ListedColormap(['lightblue', 'lightgreen']))
axs[0,1].scatter(data_x.iloc[:,0], data_x.iloc[:,1],c=data_y.values[:, 0], s=1000, cmap = ListedColormap(['blue', 'green']))
axs[0,1].set(xlabel='Gender',ylabel='Quantity', title="Predicción lineal")
print("Modelo Lineal")
print("Acuraccy: ", metrics.accuracy_score(data_y,pred))
print(metrics.confusion_matrix(data_y,pred))  
print(metrics.classification_report(data_y,pred))

#Radial
model = SVC(kernel='rbf').fit(data_x,data_y)
pred = model.predict(data_x)
X1, X2 = np.meshgrid(np.arange(data_x.values[:, 0].min() - 1,data_x.values[:, 0].max() + 1),np.arange(data_x.values[:, 1].min() - 1,data_x.values[:, 1].max() + 1))
axs[1,0].contourf(X1, X2, model.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),alpha = 0.75, cmap = ListedColormap(['lightblue', 'lightgreen']))
axs[1,0].scatter(data_x.iloc[:,0], data_x.iloc[:,1],c=data_y.values[:, 0], s=1000, cmap = ListedColormap(['blue', 'green']))
axs[1,0].set(xlabel='Gender',ylabel='Quantity', title="Predicción Radial")
print("Modelo Radial")
print("Acuraccy: ", metrics.accuracy_score(data_y,pred))
print(metrics.confusion_matrix(data_y,pred))  
print(metrics.classification_report(data_y,pred))

#Sigmoidal
model = SVC(kernel='sigmoid').fit(data_x,data_y)
pred = model.predict(data_x)
X1, X2 = np.meshgrid(np.arange(data_x.values[:, 0].min() - 1,data_x.values[:, 0].max() + 1),np.arange(data_x.values[:, 1].min() - 1,data_x.values[:, 1].max() + 1))
axs[1,1].contourf(X1, X2, model.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),alpha = 0.75, cmap = ListedColormap(['lightblue', 'lightgreen']))
axs[1,1].scatter(data_x.iloc[:,0], data_x.iloc[:,1],c=data_y.values[:, 0], s=1000, cmap = ListedColormap(['blue', 'green']))
axs[1,1].set(xlabel='Gender',ylabel='Quantity', title="Predicción sigmoidal")
print("Modelo Sigmoidal")
print("Acuraccy: ", metrics.accuracy_score(data_y,pred))
print(metrics.confusion_matrix(data_y,pred))  
print(metrics.classification_report(data_y,pred))
plt.show()

Conclusion:
* El modelo radial obtuvo mayor número de aciertos, sin embargo el intento de predicción tiende a un 50% lo cual es similar a tirar una moneda
* El modelo radial predice de mejor manera a gatos
* El modelo lineal predice de mejor manera a perros
* El modelo lineal predice de mejor manera a perros


*El modelo polinomial fue ejecutado pero no llegaba a un resultado*